In [ ]:
#r "nuget:CardanoSharp.Wallet,0.2.0"

# Wallet Basic Overview

In this section, we're going to review how to:
 - Generate a New Mnemonic
 - Restore a Mnemonic
 - Get the Root Key for the Mnemonic
 - Derive Private and Public Keys
 - Generate an Address


## Generate a New Mnemonic 

Lets start simple. Here were just going to generate a new mnemonic.

In [ ]:
using CardanoSharp.Wallet.Enums;
using CardanoSharp.Wallet.Models.Keys;
using CardanoSharp.Wallet.Extensions;

IKeyService keyService = new KeyService();
Mnemonic mnemonic = keyService.Generate(24, WordLists.English);

Console.WriteLine($"Words: {mnemonic.Words}");
Console.WriteLine($"Entropy: {mnemonic.Entropy.ToStringHex()}");

Words: beach smoke salute physical walnut erupt exhibit aerobic soldier confirm exile west shadow canyon travel welcome hungry dad mean tomato pyramid analyst ivory rabbit


Entropy: 13799afb51ff6c99d3e021ce85dd3efcbc4c43b9e7ca6f46de27720af0109dbd


## Restoring a Mnemonic

Now that we have our mnemonic, lets pretend we are loading it into a new wallet.

Here we will take our words and "Restore" them. This will produce the same object in the end.

In [ ]:
Mnemonic restoredMnemonic = keyService.Restore(mnemonic.Words, WordLists.English);

Console.WriteLine($"Words are the same: {mnemonic.Words == restoredMnemonic.Words}");
Console.WriteLine($"Entropy are the same: {mnemonic.Entropy.ToStringHex() == restoredMnemonic.Entropy.ToStringHex()}");

Words are the same: True


Entropy are the same: True


## Get the Root Key for the Mnemonic

Generating the Root Key is the first step into our Hierarchical Deterministic(HD) Wallet. This key will let us derive any key we want and we will be able to do it over and over again... Deterministic. 

As users of wallets, we are used to seeing the famous Mnemonic Words, but we also see above that there is something called Entropy. Entropy is the bytes version of your words, and this is the piece that we use to generate our Root Key. So just note that what is really going on here is Words -> Entropy -> Root Key.

In [ ]:
using CardanoSharp.Wallet.Extensions.Models;
using CardanoSharp.Wallet.Encoding;

PrivateKey rootKey = mnemonic.GetRootKey();
string rootEncoded = Bech32.Encode(rootKey.Key, "root_xsk");
Console.WriteLine($"Bech32 Encoded Root Private Key: {rootEncoded}");

Bech32 Encoded Root Private Key: root_xsk17rd2myytctkf3eq7dyeykmt2zx7nhjw6juajv8p890dpgqj4te8c6uvuql97jdyzums00ay0wnfgt6msdw5r4j3x2pdj3wrlaf94efck6mvpn


A few points about this root key before we move on to deriving various Private and Public Keys.
 -  This is the root's private key, but the private key is more specifically an extended private key. This is defined in BIP 32.
    -  Learn more about BIP 32: [Click Here](https://en.bitcoin.it/wiki/BIP_0032)
 -  When encoding, we are following the CIP 005 prefix rules. This is why we put `root_xsk`. 
    -  Learn more about CIP 0005: [Click Here](https://github.com/cardano-foundation/CIPs/blob/master/CIP-0005/CIP-0005.md)


## Derive Private and Public Keys

Now that we can Create/Retore Mnemonics and get a Root Key, it's time to start deriving some Private and Public keys. This where we dive into the Hierarchical piece of the HD wallet. 